# Classify price movements based on candlesticks statistics

This reseach comes from [this](https://www.forexfactory.com/thread/post/14707863#post14707863) post on ForexFactory. 

### Step 1: gather data and create a rolling TF

In [ ]:
# This allow jupiter to upload in real time externally modified code
%load_ext autoreload
%autoreload 2 

from x_CLASSES.download_data import DownloadData
import os
import pandas as pd
import numpy as np

In [ ]:
start_date = "01-12-2023"
end_date = "24-12-2023"
timeframe = 'tick'

csv_file_path = f"{os.getcwd()}/x_DATA/{start_date}_{end_date}  {timeframe}.csv"

if os.path.exists(csv_file_path):
    df = pd.read_csv(csv_file_path)
else:
    data_folder_path = f"{os.getcwd()}/x_DATA"
    if not os.path.exists(data_folder_path):
        os.makedirs(data_folder_path)

    !wget -P ./x_DATA https://raw.githubusercontent.com/ironhak/test4anti/main/x_DATA/01-12-2023_24-12-2023%20%20tick.csv
    df = pd.read_csv(csv_file_path)

df

In [ ]:
if "Ask" in df.columns:
    df = df.drop("Ask", axis=1)

df["Close"] = df["Bid"]
df["Open"] = df["Bid"].shift(1000)  # Shift the "Bid" values 1000 rows back
df["High"] = df["Bid"].rolling(window=1000).max()  # Calculate the rolling max over the last 1000 rows
df["Low"] = df["Bid"].rolling(window=1000).min()  # Calculate the rolling min over the last 1000 rows

df = df.dropna()
df

### Step 2: Directional bias

In [ ]:
df['Open - Close'] = (df['Open'] - df['Close']).abs()
df['Upper Wick'] = (df['High'] - df[['Open', 'Close']].max(axis=1)).abs()
df['Lower Wick'] = (df[['Open', 'Close']].min(axis=1) - df['Low']).abs()

df['Bias'] = np.where(df['Open'] > df['Close'], 'Bearish', np.where(df['Close'] > df['Open'], 'Bullish', 'Doji'))

df


### Step 3: Strength of directional bias
It can be:
- weak
- medium ("normal")
- strong

I'll calculate the average candlestick like: $\frac{\text{Average Bullish candlestick OC}+\text{Average Bearish candlestick OC}}{2}$. 
Then I'll calculate the standard deviation of the bullish candles OC and bearish candles OC and then: $\frac{\text{St.dev Bullish candlestick OC}+\text{St.dev Bearish candlestick OC}}{2}$. 
And lastly I'll create a boundary around the average, meaning that:
- If a bullish candle OC is $>=\mu+\sigma$ then it's strong. 
- If a bullish candle OC is $>=\mu$ and $<\mu+\sigma$ then it's medium ("normal").
- If a bullish candle OC is $<=\mu-\sigma$ then it's weak.

In [ ]:

# Filter rows where Bias is "Bullish"
bullish_rows = df[df['Bias'] == 'Bullish']
bearish_rows = df[df['Bias'] == 'Bearish']

# average bullish and bearish candle OC
average_bullish_diff = bullish_rows['Open - Close'].mean()
average_bearish_diff = bearish_rows['Open - Close'].mean()
mu = ( average_bullish_diff + average_bearish_diff ) / 2

# stdev bullish and bearish candle OC
stdev_bullish_diff = bullish_rows['Open - Close'].std()
stdev_bearish_diff = bearish_rows['Open - Close'].std()
sigma = ( stdev_bullish_diff + stdev_bearish_diff ) / 2


# assign strenght bias
df['Strength'] = np.where(df['Open - Close'] >= mu + sigma, 'Strong',
                          np.where((df['Open - Close'] > mu-sigma) & (df['Open - Close'] < mu + sigma), 'Medium', 'Weak'))

df